# Assignment 1: Chinese character "detection"
LT2326, Autumn 2021

Name: Max Boholm (gusbohom)

# Introduction

This notebook cotains the code for *Assignment 1: Chinese character "detection"* for the course *Machine learning for statistical NLP: Advanced* (course code LT2326), Autumn 2021. The notebook is organized into the folowing parts:

*    Meta variables (the term *hyperparameter* is here reserved for decisions on the models), which define ... the loacation (path) of the data, ... 
*    Data preparation
*    Definition and training of two models
*    Testing and evaluation
*    ...




# TO DO

**THINK AGAIN ABOUT WHAT THE PROBLEM IS**
How do you go from the "topographic" representation back to coordinates?
Consider using coordinates as targets, how do you calculate the loss? (Perhaps not that hard... one output for every dimension of coordinates... but then: ) How to handle variable length of boxes in image?

???
predict coordinates
for a 2048 x 2048 vector translate those coordinates to 0s and 1s
compute loss for that vector



*    adjusted box vs polygon proper - how to implement the latter? Cross-secting pixels? Too complicated?!
*    Rescaling - how stupid is my approach?
*    How to build (convolutional) models?
    *    Does I have to consider padding? (If so, perhaps best to use a `torch` dataloader)
    *    Flattening
*    How to visualize?
    *    Reverse flattening
    *    Heatmap simple
    *    Heatmap-layer
*    Evaluation metrics

# Libraries

In [1]:
import glob
import json
import random
from PIL import Image
import numpy as np

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Meta variables

In [2]:
#path = "../../scratch/lt2326-h21/a1/"
path = "../develop_util/"
images_dir = path + "images/"      # the directory of images to be data
meta_ctw   = path + "info.json"   # the (path to) the general meta file of CTW 
meta_train = path + "train.jsonl" # the (path to) the file containing the annotations of CTW training data

train_proportion = 0.7 # the proportion of training data; proportion of test data will be the complement of this number

#device = torch.device('cuda:0')
device = torch.device('cpu')

path_to_save_models = "../models/"

restriction = None # set to an integer, if subsample of the data is to be used in e.g. the development phase
rescale_input_to = 100 # ...
rescale_output_to = 256 # ...




In [3]:
def devider(n):
    numerator = 2
    while n > 1:
        n = n/numerator
        print(n, numerator)
        numerator = numerator * 2
devider(2048)

1024.0 2
256.0 4
32.0 8
2.0 16
0.0625 32


## Part 1: data preparation

*Decision:* batching is kept as list until feeding it to model (`flat_batch`). The list makes the evaluation part simpler as we do not have to bend our minds around multi-dimensional tensors. 

In [4]:
def only_train(file_dir = images_dir, meta_file = meta_ctw):
    """Tests whether image files in a directory is part of the training set of the CTW dataset, 
       as defined by the json metafile for CTW. Returns a list of filenames such that they are both
       in the specified directory and the CTW training set.
    """
    meta = json.load(open(meta_file,"r"))
    
    train_files_CTW = [entry["file_name"] for entry in meta["train"]]
    
    files_to_keep = []
    
    potential_files=[file.split("/")[-1] for file in glob.glob(file_dir+"*.jpg")]
    
    for file in potential_files:
        if file in train_files_CTW:
            files_to_keep.append(file)
            
    return files_to_keep

def CTW_mapper(files, meta = meta_train):
    """ Identifies annotations for files from the training set of the CTW dataset. 
        Returns pyhoton dictionary that maps filenames (keys) with annotations (values), 
        which like in the original format is a list of lists of json elements / python dictinaries. 
    """
    mapping = {}
   
    with open(meta, "r") as f:
        
        annotations_data = [json.loads(line) for line in f.readlines()]
        
        #print(annotations_data[1])

        for file in files:
            for annotation in annotations_data:
                if annotation["image_id"] == file[:-4]:
                    mapping[file] = annotation["annotations"]
                    break
    
    return mapping
                    
def img2array(file, directory = images_dir, rescale = rescale_input_to):
    """ Takes a filename of an image in a directory and returns a ...
    """
    
    img = Image.open(directory+file)
    
    if rescale != None:
        img = img.resize((rescale, rescale))
    
    img_np = np.array(img)
    
    return img_np

#Standardization is a problem, since it cannot be computed for individual images, but only for the dataset!

def coordinates2array(file, mapping, height  = 2048, width   = 2048, rescale = rescale_output_to): 
    """ Builds a vector of 0s and 1s based on the coordinates ...
    """
    
    if rescale != None:
        rescale_by = rescale / height # must come first
        height = rescale
        width = rescale
    
    char_matrix = np.zeros((height, width))
    
    char_areas = []
    for block in mapping[file]:
        for character in block:
            if character["is_chinese"] == True:
                xmin = int(character["adjusted_bbox"][0])
                ymin = int(character["adjusted_bbox"][1])
                w    = int(character["adjusted_bbox"][2])
                h    = int(character["adjusted_bbox"][3])
                
                if rescale != None:
                    xmin = int(xmin * rescale_by)
                    ymin = int(ymin * rescale_by)
                    w    = int(w    * rescale_by)
                    h    = int(h    * rescale_by)
                
                char_areas.append((xmin, ymin, w, h))
    
    for xmin, ymin, w, h in char_areas:
        
        r1 = height - ymin - h
        r2 = r1 + h
        c1 = width - xmin
        c2 = c1 + w
        
        char_matrix[r1:r2 , c1:c2] = 1
        
    return char_matrix
        
def data_builder(files, directory, mapping, restriction = restriction):
    """ Compiles the data set for use. Returns a list of dictionaties, specifying:
        -  the filename; key: "file"
        -  the training data (vector); key: "img_vector"
        -  the labels (a vector of 0s and 1s indicating boxes of characters in images); key: "label" 
    """
    
    data = []
    
    for file in files: 
        instance = {}
        instance["file"] = file
        instance["img_vector"] = img2array(file)
        instance["label"] = coordinates2array(file, mapping)
        data.append(instance) 
    
    if restriction != None:
        random.shuffle(data)
        data = data[:restriction]
        
    return data

def standardizer(dataset, scaler = StandardScaler()):
    """
    """
        
    std_data = []
    #N = len(dataset)
    example = dataset[0]["img_vector"]
    x = example.shape[0]
    y = example.shape[1]
    z = example.shape[2]
    n_features = example.size # ... or x * y * z
    
    for instance in dataset:
        std_data.append(instance["img_vector"].reshape(n_features))
    
    scaled_data = scaler.fit_transform(std_data)
    
    for i, scaled_ins in enumerate(scaled_data):
        dataset[i]["img_vector"] = scaled_ins.reshape(x, y, z)

def numpy2torch(dataset, device = device, permute = True):
    """
    """
    
    print("Shape of vector before: ", dataset[0]["img_vector"].shape)
    
    for instance in dataset:
        if permute == True: # ... hmmm 
            instance["img_vector"] = torch.Tensor(instance["img_vector"], device = device).permute(2,0,1)
        else:
            instance["img_vector"] = torch.Tensor(instance["img_vector"], device = device)
        instance["label"] = torch.Tensor(instance["label"], device = device)
    
    print("Shape of vector after: ", dataset[0]["img_vector"].shape)


In [5]:
files = only_train()
mapping = CTW_mapper(files)
my_data = data_builder(files, images_dir, mapping)
standardizer(my_data)
numpy2torch(my_data)

Shape of vector before:  (100, 100, 3)
Shape of vector after:  torch.Size([3, 100, 100])


In [6]:
def split(data       = my_data, 
          train_prop = train_proportion, 
          val_prop   = None):
    
    if val_prop != None:
        train_to_idx = int(len(data) * train_prop)
        val_to_idx   = int(len(data) * val_prop) + train_to_idx
        train = data[:train_to_idx]
        val   = data[train_to_idx:val_to_idx]
        test  = data[val_to_idx:]
        return train, val, test
    else:
        train_to_idx = int(len(data) * train_prop)
        train = data[:train_to_idx]
        test  = data[train_to_idx:]
        return train, test

train_set, test_set = split(my_data) #val?

In [7]:
# do we need padding?!
# permutation? will affect the tensor --> array --> img 

#to be used in training below ... 
def dataloader(data, batch_size=None):
    
    random.shuffle(data)
    
    if batch_size != None:
        for group in [data[i : i+batch_size] for i in range(0, len(data), batch_size)]:
            files = []
            img_vecs = []
            labels = []

            for instance in group:
                files.append(instance["file"])
                img_vecs.append(instance["img_vector"])
                labels.append(instance["label"])

            batch = {"file":files, 
                     "img_vector":img_vecs, 
                     "label":labels}
        yield batch

    else:
        for instance in data:
            yield instance


In [33]:
for i in range(3):
    print("-"*10)
    for j in dataloader(my_data, 4):
        print(j["file"])

----------
['0000181.jpg', '0000174.jpg']
----------
['0000188.jpg', '0000189.jpg']
----------
['0000181.jpg', '0000187.jpg']


### Some testing

In [ ]:
#print(files)
#print(mapping)
print(my_data[0])

In [ ]:
# Testing for 1s
labels = [x["label"] for x in my_data]
i=0
j=1
for vec in labels[:3]:
    print(j)
    for x in vec:
        for y in x:
            if y != 0:
                i = i+1
    j+=1
print(i)

### Development area

In [ ]:
import torch
t = torch.zeros(10, 10)
t[1:3, 5:7]=1 # rows, columns
print(len(t.shape))
t

In [ ]:
import numpy as np
a = np.zeros((10, 10))
a[1:3, 5:7]=1 # rows, columns
print(a.shape)
a

In [ ]:
import numpy as np
a = np.random.randn(5,25)
print(a.shape)
print(a.size)
c=np.random.randn(10)
print(c.shape)
b=a.reshape(125)
print(b.shape)

In [ ]:
from PIL import Image
import torch
import numpy as np
img = Image.open("../develop_util/images/0000172.jpg")

img_np=np.array(img)
img_vec=torch.LongTensor(img_np)
#print(img)
img_vec

In [ ]:
# testing flattening and stacking
import torch
A=[[1,2,3], [4,5,6], [7,8,9]] #three pixel img
B=[[10,11,12], [13,14,15], [16,17,18]] #another one
T1=torch.tensor(A)
T2=torch.tensor(B)
C=[T1, T2] # a batch
T3=torch.stack([torch.flatten(x) for x in C])
print("Before:")
print(C)
print("After:")
print(T3.shape)
print(T3)

print(T3.size()[0]*T3.size()[1])

## Part 2: the models

### General traing procedure

In [8]:
def flat_batch(batch):
    """Takes a list of length B of more-than-one dimensional tensors (N, M, ...) and returns a tensor of shape:
    (B, M*N*...)..."""
    
    return torch.stack([torch.flatten(instance) for instance in batch])

# do we need device assignation here?
    

In [9]:
def trainer(model, # Must be an instance of a model!
            name_of_model,
            learning_rate,
            epochs,
            batch_size,
            train_data = train_set,
            val_data = None,
            save_model = False,
            path_for_saving_model = "models/",
            my_optimizer = optim.Adam,
            my_loss_function = nn.BCELoss()):
    
    
    optimizer = my_optimizer(model.parameters(), lr=learning_rate)    
    
    #model = my_model
    model.to(device)
    model.train()
    
    loss_function = my_loss_function
    
    total_loss = 0
    
    for epoch in range(epochs):
        iterator = dataloader(train_set, batch_size)
        for i, batch in enumerate(iterator):
            
            output = model(flat_batch(batch["img_vector"]))
            targets = flat_batch(batch["label"])
            
            loss = loss_function(output, targets)
            
            total_loss += loss.item() 
            print("Epoch: ", epoch+1, "Batch: ", i, "Total loss: ", total_loss/(i+1), end='\r') 
            loss.backward() # compute gradients
            optimizer.step() # update parameters
            optimizer.zero_grad # reset gradients
            
        print()
            
        if val_data != None:
            model.eval()
            
            # DO SOMETHING
            
            model.train()
            
    if save_model == True:
        torch.save(model, path_for_saving_model+name_of_model)

### Model 0: Simple Model

#### Model defintion

In [14]:
class SimpleModel(nn.Module):
    def __init__(self, inp, hidden, outp):  
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(inp, hidden)
        self.classifier = nn.Linear(hidden, outp)
  
    def forward(self, batch): #batch is list :( Covert a list of vectors to a Plus 1 dim vector... Consider Asaads solution
        
        #flat = flat_batch(batch)
        
        compression = F.relu(self.layer1(batch))
        
        output = torch.sigmoid(self.classifier(compression))
        
        return output

#### Training and saving model

In [15]:
shape_of_input = my_data[0]["img_vector"].shape
input_size = shape_of_input[0] * shape_of_input[1] * shape_of_input[2]
shape_of_output = my_data[0]["label"].shape
output_size = shape_of_output[0] * shape_of_output[1]

print(shape_of_input)
print(shape_of_output)

my_simple_model = SimpleModel(inp=input_size, hidden=100, outp=output_size)

torch.Size([3, 100, 100])
torch.Size([256, 256])


In [16]:
my_simple_model.parameters

<bound method Module.parameters of SimpleModel(
  (layer1): Linear(in_features=30000, out_features=100, bias=True)
  (classifier): Linear(in_features=100, out_features=65536, bias=True)
)>

In [17]:
trainer(my_simple_model, # Must be an instance of a model!
        "mamma mu",
        0.005,
        2,
        2)

Epoch:  1 Batch:  0 Total loss:  0.7011890411376953
Epoch:  2 Batch:  0 Total loss:  17.661561965942383


### Model 1: ...

In [ ]:
epochs = 3
batch_size_m1 = 1
lr_m1 = 0.01 
stride_m1 = 1
window_m1 = (1,1)
m1_name = "m1_{}{}{}".format("B", "S", "X")


#### Model definition

#### Training and saving model

### Model 2: ...

In [ ]:

batch_size_m2 = 1
lr_m2 = 0.01 
stride_m2 = 1
window_m2 = (1,1)
m2_name = "m2_{}{}{}".format("B", "S", "X")


#### Model definition

#### Training and saving model

## Part 3: testing and evaluation

### Testing: setting hyperparameters

In [ ]:
batch
stride
iterations


### Evaluation: performance of best models

In [29]:
def evaluator(model, loader = dataloader, threshold = 0.5):
    model.eval()

    for instance in loader(test_set):
        
        prediction = model(torch.flatten(instance["img_vector"]))
        truth = torch.flatten(instance["label"]).int()
        #print(truth)

        mse = F.mse_loss(prediction, truth)
        
        roundof = (prediction >= threshold).int()
        
        #print(roundof * truth)
        
        tp = sum(roundof * truth)
        #print("TP: ", tp)
        fp = sum(roundof * (~truth.bool()).float())
        #print("FP: ", fp)
        tn = sum((~roundof.bool()).float() * (~truth.bool()).float())
        #print("TN: ", tn)
        fn = sum((~roundof.bool()).float() * truth)
        #print("FN: ", fn)
        
        accuracy = (tp + tn) / (tp + fp + tn + fn)
        recall = tp / (tp + fn)
        precision = tp / (tp + fp)
        f1 = (2 * recall * precision) / (recall + precision)
        
        print(instance["file"])
        print("MSE: ", mse.item())
        print("Accuracy: ", accuracy.item())
        print("Recall: ", recall.item())
        print("Precision: ", precision.item())
        print("F1: ", f1.item())
        
        print("-"*15)
        #break
        


In [30]:
evaluator(my_simple_model)

0000187.jpg
MSE:  0.40677765011787415
Accuracy:  0.57037353515625
Recall:  0.4095238149166107
Precision:  0.0015282368985936046
F1:  0.0030451102647930384
---------------
0000189.jpg
MSE:  0.3939460813999176
Accuracy:  0.573760986328125
Recall:  0.41911765933036804
Precision:  0.002042132429778576
F1:  0.004064460750669241
---------------
0000176.jpg
MSE:  0.15138916671276093
Accuracy:  0.8443756103515625
Recall:  0.2293577939271927
Precision:  0.004959825426340103
F1:  0.009709681384265423
---------------


### Visualizations